In [34]:
import pandas as pd

fire_data = pd.read_csv(r'C:\Users\natal\Desktop\IRC\IRC-Group5\IRC-Group5\fire_data8423.csv')
fire_data = fire_data.dropna()
pivoted_weather= pd.read_csv(r'weather_pivoted.csv')
# Drop columns if they exist
columns_to_drop = ['CONT_DATE', '_id']
fire_data = fire_data.drop(columns=[col for col in columns_to_drop if col in fire_data.columns])
#try merging with unpivoted index weather data, tall and skinny data

print(pivoted_weather['date'].dtype)
print(fire_data['ALARM_DATE'].dtype)


object
object


In [35]:

# Convert 'date' column to datetime
pivoted_weather['date'] = pd.to_datetime(pivoted_weather['date'])
fire_data['ALARM_DATE'] = pd.to_datetime(fire_data['ALARM_DATE'])

#Filter rows between 1984-01-01 and 2023-12-31
pivoted_weather = pivoted_weather[(pivoted_weather['date'] >= '1984-01-01') & (pivoted_weather['date'] <= '2023-12-31')]
fire_data = fire_data[(fire_data['ALARM_DATE'] >= '19840101') & (fire_data['ALARM_DATE'] <= '20231231')]



In [36]:
# Rename columns to match index names before setting index
pivoted_weather.rename(columns={'id': 'weatherstn', 'date': 'ALARM_DATE'}, inplace=True)

# Set index before merging (Speeds up join operations)
pivoted_weather.set_index(['weatherstn', 'ALARM_DATE'], inplace=True)
fire_data.set_index(['weatherstn', 'ALARM_DATE'], inplace=True)

# Perform index-based merge
pivotedweather_fire_merge = pivoted_weather.merge(fire_data, left_index=True, right_index=True, how='left')

# Reset index if needed
pivotedweather_fire_merge.reset_index(inplace=True)

To get only days with fire

In [38]:
pivotedweather_fire_merge_reg= pivotedweather_fire_merge[pivotedweather_fire_merge['Shape__Area'].notna()]

In [39]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Generating the regression df by dropping rows with NaN in 'Shape__Area'
pivot_regression_ver = pivotedweather_fire_merge_reg.fillna(0)



In [48]:


# Selecting the features for X (categorical 'obs' and numerical 'obs_value')
X = pivot_regression_ver.loc[:,'ACMH':'WT22']
# Define the target variables (multiple columns if necessary)
Y = pivot_regression_ver[['GIS_ACRES', 'Shape__Length','Shape__Area']]  # Multiple target columns

print(X.shape)
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X)  # Assume 'obs' is the categorical column




# Optional: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)


(2979, 57)


In [50]:
#training and testing datasets split
from sklearn.model_selection import train_test_split
# Splitting the dataset into training and testing set (80/20)
X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y, test_size = 0.2, random_state = 42)
# Initialize the RandomForestRegressor
model = RandomForestRegressor(n_estimators=50, random_state=42, oob_score=True)


model.fit(X_train,Y_train)

c:\Users\natal\anaconda3\Lib\site-packages\sklearn\utils\_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


RandomForestRegressor(n_estimators=50, oob_score=True, random_state=42)

In [51]:
# Make predictions
Y_pred = model.predict(X_test)

# Evaluate model performance
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")
print(f"OOB Score: {model.oob_score_}")

Mean Squared Error: 1754470232327146.2
R-squared: -0.6942284880543953
OOB Score: -0.04685771879424094
